In [ ]:
#!pip install emoji

In [ ]:
# Import Required Libraries
import pandas as pd
pd.set_option('display.max_columns', 20)
import re
import emoji
import numpy as np
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
from google.colab import files


In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [3]:
!unzip -uq /content/drive/MyDrive/combined_data.zip

In [4]:
def preprocess_text_for_summarization(text):
    # Convert to lowercase
    text = text.lower()

    # Remove HTML tags
    text = re.sub(r'<[^>]*>', '', text)

    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Remove mentions
    text = re.sub("@[A-Za-z0-9_]+", "", text)

    # Remove hashtags
    text = re.sub("#[A-Za-z0-9_]+", "", text)

    # Remove punctuations
    text = re.sub('[^a-zA-Z]', ' ', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove emojis
    text = emoji.demojize(text)

    # Tokenization
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    tokens = [token for token in tokens if token.lower() not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token, pos='v') for token in tokens]

    # Join tokens back to form preprocessed text
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text


In [6]:
text_columns = ['article', 'summary']

chunk_reader = pd.read_csv('/content/combined_data.csv', chunksize=20000)

cleaned_chunks = []

for chunk in chunk_reader:
    for col in text_columns:
        preprocessed_text = chunk[col].apply(preprocess_text_for_summarization)
        chunk[col] = preprocessed_text

    cleaned_chunks.append(chunk)

df_cleaned = pd.concat(cleaned_chunks, ignore_index=True)

In [7]:
df_cleaned.head()

,article,summary
0,sally forrest actress dancer grace silver scre...,sally forrest actress dancer grace silver scre...
1,middle school teacher china ink hundreds sketc...,work include picture presidential palace yangt...
2,man convict kill father sister former girlfrie...,iftekhar murtaza convict year ago kill ex girl...
3,avid rugby fan prince harry could barely watch...,prince harry attendance england crunch match f...
4,triple radio producer inundate message prospec...,nick slater colleagues upload picture facebook...


In [8]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314196 entries, 0 to 314195
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   article  314196 non-null  object
 1   summary  314196 non-null  object
dtypes: object(2)
memory usage: 4.8+ MB


In [9]:
df_cleaned.isnull().sum()

article    0
summary    0
dtype: int64

In [10]:
df_cleaned.duplicated().sum()

3245

In [11]:

df_cleaned.drop_duplicates(inplace=True)

df_cleaned.duplicated().sum()

0

In [14]:
df_cleaned.to_csv('df_cleaned.csv', index=False)

In [15]:

files.download('df_cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>